<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [3]:
import numpy as np
class PuzzleNode():
    """
    Class PuzzleNode: Provides a structure for performing A* search
    for the n^2-1 puzzle.
    
    Attribute note: 
        Takes the state as any input shape. This is later converted
        to valid input (array) in the solvePuzzle function. 
    """
    def __init__(self, state, gval = 0, fval = 0, parent = None):

        self.gval = gval    # current path cost
        self.fval = fval    # path cost + estimated heuristic cost
        self.parent = parent  # parent for retrieving final solution
        self.state = state   # state of the puzzle in the form of array
        
        self.length = len(state)
        self.pruned = False   # tells if node can be ignored during search

    def __str__(self):
        """Returns a n-by-n board with allignment."""
        return('\n'.join(' '.join('{:{w}d}'.format(i, w=len(str(self.length**2-1)))
                                  for i in row) for row in self.state))        
 
    def __lt__(self, other):
        """Comparing states in the search tree."""
        return self.fval < other.fval

In [4]:
test_array = np.array([[0,1,2], [3,4,5], [6,7,8]])
node = PuzzleNode(test_array)
print(node)

0 1 2
3 4 5
6 7 8


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [5]:
#Concept adapted from https://www.youtube.com/watch?v=6TsL96NAZCo&ab_channel=JohnLevine

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    h = 0
    counter = 0
    
    for i in range(len(state)): 
        for j in range(len(state[0])): 
            if counter != 0 and state[i][j] != counter: 
                h += 1
            counter += 1 
    return h 

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given 
    the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    Method: 
      First, a completed state is generated. Then, for each tile in the current
      state, the distance in x and y values from the goal state is calculated and 
      summed, giving the total Manhattan distance. 
    """
    n_tiles = (len(state) ** 2)
    n = len(state)
    
    state = np.array(state).reshape(n, n)
        
    cur_state = list(state.flatten())  
    goal_state = list(range(n**2))
        
    manhattan_dist = [abs(b % n - g % n) + abs(b//n - g//n) for
                      b, g in ((cur_state.index(i), goal_state.index(i)) for
                      i in range(1, n_tiles))]
        
    return sum(manhattan_dist)

#Euclidean Distance Heuristic
def h3(state): 
    """
    This function returns the euclidean distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the euclidean distance from the solved configuration
    Method: 
      A Similar method is followed to the manhattan distance, but instead of 
      summing the horizontal and vertical distances, the diagonal distance 
      is found using the pythagoran theorem. 
    """
    counter = 0
    x_axis, y_axis = len(state), len(state[0])
    euclidean_dist = 0 
    
    #Create a termination state 
    termination_state = [[0]*y_axis for row in range(x_axis)]
    for i in range(len(state)): 
        for j in range(len(state[0])):
            termination_state[i][j] = counter
            counter += 1 
        
    for i in range(len(state)): 
        for j in range(len(state[0])): 
            value = state[i][j]
            if value != 0: 
                subarray =  [x for x in termination_state if value in x][0]
                value_x_axis, value_y_axis = termination_state.index(subarray), subarray.index(value)
                add_values = ((abs(value_x_axis - i))**2 + (abs(value_y_axis - j))**2)**0.5
                euclidean_dist += add_values
        
    return euclidean_dist
    

heuristics = [h1, h2, h3]

##### Explanation
In the above code I have created three different heuristics to estimate the distance from the root node to the goal node. These heuristics are Manhattan Distance, Euclidean distance and Misplaced tiles. After the thorough examination and testing these heuristics, I concluded that neither of the other two heuristics were able to outperform Manhattan distances, despite getting the same distances. 

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

The solvability test function below tests if a given input array is solvable based on the number of inversions on the board and the board size.

- If the grid width is odd, then the number of inversions in a solvable situation is even.
- If the grid width is even, and the blank is on an even row counting from the bottom (second-last, fourth-last etc), then the number of inversions in a solvable situation is odd.
- If the grid width is even, and the blank is on an odd row counting from the bottom (last, third-last, fifth-last etc) then the number of inversions in a solvable situation is even.

In [6]:
#Concept adapted from: https://datawookie.netlify.com/blog/2019/04/sliding-puzzle-solvable/
# Extension rules from: https://www.cs.bham.ac.uk/~mdr/teaching/modules04/java2/TilesSolvability.html

def solvability_test(state):
    """
    Input:
        state (arr) Input state array as a list of lists.
    Output:
        Boolean Value
    """
    count = 0
    n = len(state)
    n_tiles = (len(state) ** 2)
    
    #reshaping the input to be of a valid format
    state = np.array(state).reshape(n, n) 
    
    loc_zero_tile = tuple(*np.argwhere(state == 0))
    array_tiles = list(state.flatten())  

    for i in range(n_tiles - 1):
        for j in range(i+1, n_tiles):
            if array_tiles[j] and array_tiles[i] and array_tiles[i] > array_tiles[j]:
#                 print(array_tiles[i], "is larger than", array_tiles[j])
                count += 1
                
    # Different scenarios for solvability.
    if (n % 2 != 0) and (count % 2 == 0):
        return(True)
    elif (n % 2 == 0) and (loc_zero_tile[0] % 2 != 0) and (count % 2 != 0):
        return(True)
    elif (n % 2 == 0) and (loc_zero_tile[0] % 2 == 0) and (count % 2 == 0):
        return(True)
    else:
        return(False)

In [7]:
# Main solvePuzzle function.
from queue import PriorityQueue


def solvePuzzle(state, heuristic = None):
    """
    This function should solve the n**2-1 puzzle for any n > 2
    (although it may take too long for n > 4)).
    
    Inputs:
        state: The initial state of the puzzle as a list of lists.
        heuristic: A handle to a heuristic function.
        [h1, h2, h3] (misplaced tiles, manhattan distance, Eucleadean Distance)
        
    Outputs:
        steps: The number of steps to optimally solve the puzzle, 
                excluding the initial state.
        expanded: The number of nodes expanded to reach the solution.
        max_frontier: The maximum size of the frontier over the search.
        optimal path: The optimal path as a list of list of lists.
                optimal_path[:,:,i] gives a list of lists that represents
                the state of the board at step i of the solution.
        err: An error code.  If state is not of the appropriate size
                and dimension, return -1. If the state is not solvable,
                returns -2.
    """
#_______________________________________________INPUT TESTS______________________________________#
    
    n = len(state)
    flattened_goal = list(range(n**2))

    if not solvability_test(state):
        print("This puzzle is not solveable.")
        return(0, 0, 0, None, -2)
    
    try:
        state = np.array(state).reshape(n, n)
        # state is now a n*n array, no longer a list of lists!
    except:
        print("The input could not be shaped correctly.")
        return(0, 0, 0, None, -1) ##If the state is not of n*n dimensions, return err = -1
    
    flattened_state = state.flatten()
    if sorted(flattened_state) != flattened_goal:
        print("Input invalid. Check your state for duplicates.")
        return(0, 0, 0, None, -1)
    
#____________________________________________SEARCH ALGORITHM_______________________________________#    

    initial_node = PuzzleNode(state, gval = 0, fval = heuristic(state))
    costs = {tuple(initial_node.state.flatten()): initial_node}

    # Frontier, stored as a Priority Queue to maintain ordering
    frontier = PriorityQueue()
    frontier.put(initial_node)

    Orthogonal_moves = [(1,0),(0,1),(-1,0),(0,-1)]
    expansion_count = 0
    max_frontier = 0

    while not frontier.empty():
        
        # retrieving worst case frontier size
        if frontier.qsize() > max_frontier:
            max_frontier = frontier.qsize()

        current_node = frontier.get()
        
        if current_node.pruned:
            continue

        # Goal check
        if list(current_node.state.flatten()) == flattened_goal:
            print("Algorithm reached the goal state in",expansion_count,
                  "steps. Max Frontier size", max_frontier )
            break
        
        # finding Zero tile
        zero_tile = tuple(*np.argwhere(current_node.state == 0))

        # expand in the orthogonal direction to generate sucessor states
        # avoiding states that would go outside the edges        
        next_moves = [tuple( [sum(i) for i in zip(zero_tile, move)] )
                      for move in Orthogonal_moves] 
        valid_moves = [i for i in next_moves if
                       (0 <= i[0] < n and 0 <= i[1] < n)]
        

        for m in valid_moves:
            gval = current_node.gval + 1 # minimum cost for child
    
            # try the move by swapping 0 and non-0 tile
            test = np.copy(current_node.state)
            z = zero_tile
            test[z], test[m] = test[m], test[z]        

            # If move is explored, update the path & mark for removal
            if tuple(test.flatten()) in costs:
                if costs[tuple(test.flatten())].gval > gval:
                    costs[tuple(test.flatten())].pruned = True
                else:
                    continue 
            
            # get heuristic cost from the node to the end and
            # add the child to the frontier & cost database
            hcost = heuristic(test)
            next_node = PuzzleNode(test, gval = gval,
                                   fval = (gval + hcost),  parent=current_node)
            
            frontier.put(next_node)
            costs[tuple(test.flatten())] = next_node 
            
        expansion_count = expansion_count + 1

#_______________________________OPTIMAL PATH_______________________________________#

    optimal_path_nodes = []
    solution_node = current_node
    
    while solution_node:
        optimal_path_nodes.append(solution_node)
        solution_node = solution_node.parent
        
    optimal_path = []
    
    #reverses through the solution
    for j, node in reversed(list(enumerate(optimal_path_nodes))):
        slot = len(optimal_path_nodes) - 1 - j
        optimal_path.append(node.state.tolist())
    
    return (len(optimal_path_nodes)-1, expansion_count,
            max_frontier, optimal_path, 0)

#### Explanation

###### Input Tests
The first part of the code tests for correct shape of the inputs, Given an input is incorrect, we try to reshape the state from a list of lists to an array. Next we check if the input has any duplicates in it as we know there aren't any duplicates in the 8-puzzle. Once all these tests are passed, we flatten & sort the input & compare it to the goal state (for the initial test). If either of the tests fail, the input is marked as incorrect.
        
##### Search Algorithm
The second part of the code first does a set up for the upcoming search algorithm. We first set the initial node to the start state, then initiate the cost dictionary for tracking search cost which enables the algorithm as a whole to have a O(n) runtime, and a O(1) lookup time for each instance of checking if a child node has been previously initialized and stored in costs. Then we generate the frontier and count the maximum fronties size after every step. After this initial setup is done, the code then generates possible future moves and appends them to the frontier until a solution is found while the cost dictionary keeps track of previously explored states.
    
##### Optimal Path 
The last part of the code reconstructs the path of the optimal solution and returns summary statistics of the algorithm to evaluate performance.

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [8]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

Input invalid. Check your state for duplicates.


In [9]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [10]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


Algorithm reached the goal state in 684 steps. Max Frontier size 430
Algorithm reached the goal state in 121 steps. Max Frontier size 68
Algorithm reached the goal state in 27257 steps. Max Frontier size 12623
Algorithm reached the goal state in 1740 steps. Max Frontier size 981
Algorithm reached the goal state in 53225 steps. Max Frontier size 20172
Algorithm reached the goal state in 1648 steps. Max Frontier size 940


In [11]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

Algorithm reached the goal state in 9 steps. Max Frontier size 13
Algorithm reached the goal state in 9 steps. Max Frontier size 13


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [12]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

This puzzle is not solveable.


In [13]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

Algorithm reached the goal state in 159 steps. Max Frontier size 91
Algorithm reached the goal state in 121 steps. Max Frontier size 68
Algorithm reached the goal state in 3624 steps. Max Frontier size 1999
Algorithm reached the goal state in 1740 steps. Max Frontier size 981
Algorithm reached the goal state in 8732 steps. Max Frontier size 4554
Algorithm reached the goal state in 1648 steps. Max Frontier size 940


AssertionError: 

In [ ]:
## Memoization test - will be carried out after submission

#### References
Anonymous. (n.d.). Cooperating Intelligent Systems - Uniformed search. Retrieved from https://www.dcc.fc.up.pt/~ines/aulas/1213/SI/AIMA_ch3_L2-complement.ppt

Jason, Y. (n.d.). Linear Distance Calculator . Retrieved October 26, 2019, from https://github.com/Jason-Yuan/8PuzzleGameSovler/blob/master/heuristic_estimate.py.Stack Overflow . (2011, March 1).

Rotating a two-dimensional array in Python. Retrieved October 26, 2019, from https://stackoverflow.com/questions/8421337/rotating-a-two-dimensional-array-in-python.Yar Kan, A. (2016, May 3).

Implementing A-star(A*) to solve N-Puzzle. Retrieved from https://algorithmsinsight.wordpress.com/graph-theory-2/a-star-in-general/implementing-a-star-to-solve-n-puzzle/.

